# Opis notatnika

### Importy

In [ ]:
import sys
from pathlib import Path
import pandas as pd

SRC = (Path("..")).resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))


In [ ]:
from classes.pokemon_team import PokemonTeam
from visualization.utils import summarize
from data.data import get_pokemons
from report.reports import PdfReport
from experiments.sa_experiments import run_multiple_runs, compare_to_hill_climb, compare_to_random_search
from visualization.plots import visualize_opponents_typing_distribution, visualize_opponents_stat_sums_violin
from constants import REPORT_DIR

In [ ]:
from solvers import EvolutionaryAlgorithmPokemonSolver

### Funckje pomocnicze

In [ ]:
import numpy as np
import pandas as pd
from pandera.typing import DataFrame

def ea_run_multiple_runs_df(
    pokemons: DataFrame[PokemonSchema],
    solver: EvolutionaryAlgorithmPokemonSolver,
    runs: int = 8,
    seed_base: int | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      df_runs: one row per run with best team info
      df_summary: aggregated stats (mean/median/std/min/max/count)
    """
    rows = []

    for i in range(runs):
        if seed_base is not None and hasattr(solver, "seed"):
            # jeśli solver ma seed (u Ciebie EA go nie ma, ale zostawiam kompatybilnie)
            solver.seed = seed_base + i

        best_team, best_fitness, _, _ = solver.solve(pokemons)

        rows.append(
            {
                "run": i,
                "solver": "EA",
                "fitness": float(best_fitness),
                "stats_sum": int(best_team.get_stats_sum()),
                "team_signature": "|".join(best_team.get_ids()),
                "team_repr": repr(best_team),
            }
        )

    df_runs = pd.DataFrame(rows)

    df_summary = (
        df_runs.groupby("solver")["fitness"]
        .agg(["mean", "median", "std", "min", "max", "count"])
        .reset_index()
        .sort_values("mean", ascending=False)
        .round(6)
    )

    return df_runs, df_summary
